In [1]:
best_prec1 = 0
evaluate = True
import time
import torch
import torch.nn as nn
from ptflops import get_model_complexity_info
import torch.nn.functional as F
import torch.nn.init as init
import os
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import numpy as np
import math
from torch.nn.parameter import Parameter
import wandb
import math
from models import *

def main():
    global best_prec1, evaluate
    

    wandb.init(project='DenseNet', entity='zhdwu_1989')
   ################################################################################################model

    model = densenet121()
    wandb.watch(model)
        

    with torch.cuda.device(0):
        flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
        print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
        print('{:<30}  {:<8}'.format('Number of parameters: ', params))

    wandb.config.update({"Parameters": params, "FLOPs": flops})
    


    model.cuda()

    cudnn.benchmark = True

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=128, shuffle=True,
        num_workers=4, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(model.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=5e-4)
######################################################################################更改学习率

######################################################################################更改学习率
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=250)


    for epoch in range(0, 250):

        # train for one epoch
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        wandb.log({'lr': optimizer.param_groups[0]['lr']})
        train(train_loader, model, criterion, optimizer, epoch)
#########################################################################################更改学习率
        lr_scheduler.step()

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion, epoch)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        if epoch > 0 and epoch % 20 == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, is_best, filename=os.path.join('./', 'vanilla_checkpoint.th'))

        save_checkpoint({
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, filename=os.path.join('./', 'vanilla_model.th'))
    ##########################################################################保存模型
    model.save("ReLU.h5")


    wandb.run.finish()



def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = input.cuda()
        target_var = target

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 20 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))


def validate(val_loader, model, criterion, epoch):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()


            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    wandb.log({
    'epoch': epoch,
    'Top-1 accuracy': top1.avg,
    'loss': losses.avg,
    })

    return top1.avg

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [2]:
if __name__ == '__main__':
    main()

wandb: Currently logged in as: zhdwu_1989 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/ubuntu/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


DenseNet(
  7.979 M, 100.000% Params, 0.06 GMac, 100.000% MACs, 
  (features): Sequential(
    6.954 M, 87.154% Params, 0.058 GMac, 98.278% MACs, 
    (conv0): Conv2d(0.009 M, 0.118% Params, 0.002 GMac, 4.047% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(0.0 M, 0.002% Params, 0.0 GMac, 0.055% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act0): Act_op(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
    (pool0): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 0.028% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      0.335 M, 4.199% Params, 0.021 GMac, 36.029% MACs, 
      (denselayer1): _DenseLayer(
        0.045 M, 0.570% Params, 0.003 GMac, 4.886% MACs, 
        (norm1): BatchNorm2d(0.0 M, 0.002% Params, 0.0 GMac, 0.014% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): Act_op(0.0 M, 0.000% Params, 0.0 GMac,

Files already downloaded and verified
current lr 1.00000e-02
Epoch: [0][0/391]	Time 1.190 (1.190)	Data 0.143 (0.143)	Loss 6.9947 (6.9947)	Prec@1 0.000 (0.000)
Epoch: [0][20/391]	Time 0.049 (0.106)	Data 0.000 (0.007)	Loss 3.4849 (5.4110)	Prec@1 13.281 (10.975)
Epoch: [0][40/391]	Time 0.055 (0.080)	Data 0.000 (0.004)	Loss 2.0845 (3.9084)	Prec@1 20.312 (14.787)
Epoch: [0][60/391]	Time 0.049 (0.071)	Data 0.000 (0.002)	Loss 1.9143 (3.2944)	Prec@1 37.500 (18.417)
Epoch: [0][80/391]	Time 0.054 (0.067)	Data 0.000 (0.002)	Loss 1.8462 (2.9590)	Prec@1 32.031 (20.737)
Epoch: [0][100/391]	Time 0.050 (0.064)	Data 0.000 (0.002)	Loss 1.8615 (2.7356)	Prec@1 32.031 (23.035)
Epoch: [0][120/391]	Time 0.050 (0.062)	Data 0.000 (0.001)	Loss 1.7142 (2.5880)	Prec@1 33.594 (24.490)
Epoch: [0][140/391]	Time 0.049 (0.061)	Data 0.000 (0.001)	Loss 1.8381 (2.4793)	Prec@1 31.250 (25.654)
Epoch: [0][160/391]	Time 0.049 (0.060)	Data 0.000 (0.001)	Loss 1.7666 (2.3889)	Prec@1 28.125 (26.674)


KeyboardInterrupt: 